In [3]:
import os
from os import path
import shutil
import pandas as pd
import random
from tqdm.notebook import tqdm

In [4]:
df_corpus = pd.read_csv("/home/crazyjeannot/Documents/doctorat/data/data_chapitres/metadata/corpus.csv")
path_to_txt = "/home/crazyjeannot/Documents/doctorat/corpora/corpus_chapitres_txt/"
path_to_chunks_txt = "/home/crazyjeannot/Documents/doctorat/corpora/corpus_chapitres_chunks_txt/"

In [3]:
df_corpus.head(1)

,doc_name,date,auteur,titre,fabula,genre
0,1861_Segur-comtesse-de_Pauvre-Blaise,1861,comtesse de segur,pauvre blaise,False,littérature jeunesse


In [21]:
def select_and_copy_novels(N, path_to_folder, metadata_df):
    # Create a new folder to store selected novels
    new_folder_path = os.path.join(os.getcwd(), '../corpus/random_txt')
    os.makedirs(new_folder_path, exist_ok=True)

    # Filter novels based on subgenre
    filtered_novels = metadata_df[(metadata_df['genre'] != "roman d'aventures") & (metadata_df['genre'] != "roman historique") & (metadata_df['date'] >= 1840) & (metadata_df['date'] <= 1935)]

    # Randomly select N novels
    selected_novels = random.sample(filtered_novels['doc_name'].tolist(), N)

    # Copy selected novels to the new folder
    for novel in selected_novels:
        novel_path = os.path.join(path_to_folder, f"{novel}.txt")
        destination_path = os.path.join(new_folder_path, f"{novel}.txt")
        shutil.copy2(novel_path, destination_path)

    return selected_novels, new_folder_path

In [22]:
selected_novels, new_folder_path = select_and_copy_novels(102, path_to_txt, df_corpus)

In [27]:
def create_metadata_dataframe(path_to_aventures, path_to_random):
    aventures_files = [f.replace('.txt', '') for f in os.listdir(path_to_aventures) if f.endswith('.txt')]
    random_files = [f.replace('.txt', '') for f in os.listdir(path_to_random) if f.endswith('.txt')]

    aventures_df = pd.DataFrame({'doc_name': aventures_files, 'subgenre': 'aventure'})
    random_df = pd.DataFrame({'doc_name': random_files, 'subgenre': 'random'})

    metadata_df = pd.concat([aventures_df, random_df], ignore_index=True)

    # Save the DataFrame to a CSV file
    metadata_df.to_csv('../data/metadata/metadata.csv', index=False)

    return metadata_df

In [28]:
df_meta = create_metadata_dataframe("/home/crazyjeannot/Documents/doctorat/AVENTURES/corpus/aventures_txt/", "/home/crazyjeannot/Documents/doctorat/AVENTURES/corpus/random_txt/") 

# Chunkise Texts

In [6]:
def chunkize_text_files(N, M, path_to_folder, new_dir):
    # Create a new directory if it doesn't exist
    os.makedirs(new_dir, exist_ok=True)

    # Iterate through each text file in the given folder
    for filename in tqdm(os.listdir(path_to_folder), total=len(os.listdir(path_to_folder))):
        if filename.endswith(".txt"):
            file_path = os.path.join(path_to_folder, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                tokens = content.split()

                # Create overlapping chunks
                for i in range(0, len(tokens) - N + 1, M):
                    chunk = tokens[i:i + N]

                    # Save the chunk to a new text file
                    chunk_filename = f"{filename}_chunk_{i // M}.txt"
                    chunk_filepath = os.path.join(new_dir, chunk_filename)
                    with open(chunk_filepath, 'w', encoding='utf-8') as chunk_file:
                        chunk_file.write(' '.join(chunk))
                        

In [7]:
chunkize_text_files(3000, 1000, path_to_txt, path_to_chunks_txt)

  0%|          | 0/2954 [00:00<?, ?it/s]

# SELECT N RANDOM CHUNKS

In [26]:
path_to_N_random_chunks = "/home/crazyjeannot/Documents/doctorat/AVENTURES/data/10000_random_chapitres_chunks_txt/"

In [11]:
def random_copy_files(source_folder, destination_folder, num_files=10000):
    # Get a list of all files in the source folder
    all_files = os.listdir(source_folder)

    # Ensure that the number of files to copy is not greater than the total number of files in the source folder
    num_files = min(num_files, len(all_files))

    # Randomly select num_files files
    selected_files = random.sample(all_files, num_files)

    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Copy selected files to the destination folder
    for file_name in selected_files:
        source_path = os.path.join(source_folder, file_name)
        destination_path = os.path.join(destination_folder, file_name)
        shutil.copy2(source_path, destination_path)


In [13]:
random_copy_files(path_to_chunks_txt, path_to_N_random_chunks, 10000)

## Delete selected Files

In [24]:
df = pd.read_csv('GPT_1900_EMBEDDINGS_OUTPUT_TEMP.csv')
df.set_index('doc', inplace=True)

In [25]:
df

,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
doc,,,,,,,,,,,,,,,,,,,,,
1883_Guy-de-Maupassant_Une-vie.txt_chunk_31,-0.007205,0.001200,-0.001279,-0.007666,-0.014825,0.027157,0.009009,-0.018564,-0.028611,-0.032688,...,-0.008146,-0.018810,0.013501,-0.004410,-0.019355,0.006079,0.004566,-0.013267,-0.000460,-0.017824
1845_Sue-Eugene_Le-juif-errant_tome-I.txt_chunk_170,0.001785,0.002469,0.000195,-0.027150,-0.009730,0.020346,0.004892,-0.003191,-0.043001,-0.015756,...,0.015273,-0.028748,0.015474,-0.001978,-0.027540,0.022185,0.018749,-0.009616,0.006221,-0.001208
1884_Gouraud-Julie_Le-Vieux-chateau.txt_chunk_3,-0.001259,0.017542,0.013054,-0.022841,-0.011786,0.051047,0.001794,0.008532,-0.022589,-0.015856,...,-0.000055,-0.030915,0.009946,-0.004913,-0.022921,0.012370,-0.005541,-0.009130,-0.003169,-0.009807
1904_Rolland-Romain_Jean-Christophe_Tome-II-Le-matin.txt_chunk_30,-0.021479,-0.016517,0.009400,-0.001616,-0.010322,0.034839,0.003191,-0.022819,-0.018481,-0.031894,...,-0.003156,-0.024053,-0.001598,-0.002962,-0.038474,0.016172,-0.009293,-0.009944,-0.002292,-0.029240
1925_Zevaco-Michel_Le-fils-de-Pardaillan_tome-2.txt_chunk_11,-0.027041,-0.011744,0.012790,-0.013983,0.006891,0.035795,0.024292,0.000293,-0.034669,-0.032738,...,0.007420,-0.020284,-0.000696,-0.016088,-0.035500,0.022013,0.005708,-0.005544,-0.006217,-0.017737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1866_Ponson-du-Terrail-Pierre_La-Resurrection-de-Rocambole_Tome-II-Saint-Lazare-L-Auberge-maudite-La-Maison-de-fous.txt_chunk_0,-0.008235,-0.010323,0.010924,-0.017241,0.004060,0.020876,0.000949,-0.013620,-0.036860,-0.016606,...,0.008215,-0.027023,0.016592,0.001214,-0.013627,0.015295,-0.003901,0.009742,0.006009,-0.016309
1879_Cadol-Edouard_La-Diva.txt_chunk_22,-0.021541,0.002427,0.000287,-0.013817,-0.013983,0.008892,-0.012360,-0.000984,-0.032167,-0.031144,...,-0.004297,-0.007199,0.023973,-0.021348,-0.030260,0.022771,0.003283,-0.007247,-0.013493,-0.005389
1997_Ferney-Alice_Grace-et-denuement.txt_chunk_28,-0.019978,-0.004431,0.002700,-0.016053,-0.010726,0.040667,0.012142,-0.006509,-0.020346,-0.033713,...,0.007467,-0.019886,-0.005340,-0.000183,-0.034740,0.028130,0.001860,-0.012478,0.003892,-0.015922


In [27]:
def delete_files(folder_path, file_names):
    # Iterate through files in the folder
    for file_name in os.listdir(folder_path):
        file_path = path.join(folder_path, file_name)

        # Check if the file is in the list of file names (without extensions)
        if path.isfile(file_path) and path.splitext(path.basename(file_path))[0] in file_names:
            try:
                # Delete the file
                os.remove(file_path)
                print(f"File {file_name} deleted successfully.")
            except Exception as e:
                print(f"Error deleting file {file_name}: {e}")

In [ ]:
delete_files(path_to_N_random_chunks, list(df.index))